In [10]:
import semantic_kernel
print(semantic_kernel.__version__)

1.16.0


In [11]:
#!pip install semantic-kernel==1.5.0

In [23]:
#help(AzureChatCompletion)

# Full code

In [ ]:
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import (
    AzureChatCompletion, 
    AzureAISearchDataSource, 
    AzureChatPromptExecutionSettings,
    ExtraBody
)
from semantic_kernel.connectors.ai.open_ai.prompt_execution_settings.azure_chat_prompt_execution_settings import (
    AzureAISearchDataSourceParameters,
    ApiKeyAuthentication,
    AzureEmbeddingDependency,
    DataSourceFieldsMapping
)
from semantic_kernel.core_plugins import TimePlugin
from semantic_kernel.prompt_template.input_variable import InputVariable
from semantic_kernel.prompt_template.prompt_template_config import PromptTemplateConfig
from semantic_kernel.contents.chat_history import ChatHistory  # Import ChatHistory
from semantic_kernel.functions import KernelFunction  # Import KernelFunction
from semantic_kernel.functions.kernel_arguments import KernelArguments  # Import KernelArguments

def create_chatbot(api_key: str, endpoint: str, deployment_name: str, search_endpoint: str, search_api_key: str):
    """
    Create a chatbot using Semantic Kernel, Azure Chat Completion, and additional configurations.

    Args:
        api_key (str): Azure OpenAI API key.
        endpoint (str): Azure OpenAI endpoint.
        deployment_name (str): Azure deployment name for the model.
        search_endpoint (str): Azure Cognitive Search endpoint.
        search_api_key (str): API key for Azure Cognitive Search.

    Returns:
        Function: A function that interacts with the chatbot.
    """
    # Step 1: Initialize Semantic Kernel
    kernel = sk.Kernel()

    # Step 2: Configure API Key Authentication
    api_auth = ApiKeyAuthentication(api_key=api_key)

    # Step 3: Configure prompt execution settings
    execution_settings = AzureChatPromptExecutionSettings(
        temperature=0.7,  # Control randomness
        max_tokens=150,  # Limit response length
        top_p=0.9,       # Top-p sampling for diversity
        stop_sequences=["\n"]  # Define stopping points
    )

    # Step 4: Configure data source parameters for Azure Cognitive Search
    search_params = AzureAISearchDataSourceParameters(
        endpoint=search_endpoint,
        api_key=search_api_key,
        index_name="knowledgebase"  # Example index name
    )

    # Step 5: Map data source fields
    field_mapping = DataSourceFieldsMapping(
        content_field="content",  # Field containing document text
        id_field="id",           # Field containing document ID
        title_field="title"      # Field containing document title
    )

    # Step 6: Define embedding dependencies
    embedding_dependency = AzureEmbeddingDependency(
        model_name="text-embedding-ada-002",  # Example embedding model
        api_auth=api_auth
    )

    # Step 7: Configure search data source
    search_data_source = AzureAISearchDataSource(
        parameters=search_params,
        fields_mapping=field_mapping,
        embedding_dependency=embedding_dependency
    )

    # Step 8: Add search data source to the kernel
    kernel.add_ai_service("search", search_data_source)

    # Step 9: Azure Chat Completion configuration
    extra_body = ExtraBody()
    extra_body.add("user", "default-user")
    extra_body.add("conversation_id", "example-chatbot-session")

    azure_chat_completion = AzureChatCompletion(
        deployment_name=deployment_name,
        endpoint=endpoint,
        api_key=api_key,
        execution_settings=execution_settings,
        extra_body=extra_body
    )
    kernel.add_ai_service("chat", azure_chat_completion)

    # Step 10: Load TimePlugin for time-based responses
    time_plugin = TimePlugin()
    kernel.import_plugin(time_plugin, "time")

    # Step 11: Define Chatbot Prompt Template
    prompt_template_config = PromptTemplateConfig(
        inputs=[InputVariable(name="user_input", description="Input from the user")],
        description="A chatbot that provides helpful and engaging responses."
    )
    prompt_template = kernel.create_prompt_template(
        template="The user says: {{user_input}}. Respond in a helpful and concise manner.",
        config=prompt_template_config
    )

    # Step 12: Initialize ChatHistory for managing conversation context
    chat_history = ChatHistory()

    # Step 13: Define a function to interact with the chatbot
    async def chat_with_bot(user_input: str) -> str:
        """
        Interact with the chatbot.

        Args:
            user_input (str): Input message from the user.

        Returns:
            str: Response from the chatbot.
        """
        # Add the user input to chat history
        chat_history.add_user_message(user_input)

        # Process the input through Semantic Kernel
        result = await kernel.run_async(
            prompt_template, 
            user_input=user_input
        )

        # Add the chatbot's response to chat history
        chat_history.add_ai_message(result)

        # Optionally, retrieve the full conversation history if needed
        conversation = chat_history.get_history()
        print("Conversation History:", conversation)

        return result

    return chat_with_bot


# Example Usage: Main script to interact with the chatbot
if __name__ == "__main__":
    import asyncio

    # Azure configuration
    AZURE_API_KEY = "your-azure-api-key"
    AZURE_ENDPOINT = "https://your-azure-endpoint.openai.azure.com/"
    DEPLOYMENT_NAME = "your-deployment-name"

    # Azure Cognitive Search configuration
    SEARCH_ENDPOINT = "https://your-search-endpoint.search.windows.net"
    SEARCH_API_KEY = "your-search-api-key"

    # Create chatbot instance
    chatbot = create_chatbot(AZURE_API_KEY, AZURE_ENDPOINT, DEPLOYMENT_NAME, SEARCH_ENDPOINT, SEARCH_API_KEY)

    # Run chatbot interaction
    async def main():
        user_input = "What can you tell me about climate change?"
        response = await chatbot(user_input)
        print("Chatbot:", response)

    asyncio.run(main())

In [116]:
#Notes from
#https://www.youtube.com/watch?v=Am0MDRVE2v8

In [44]:
# Chat completion
#https://learn.microsoft.com/en-us/semantic-kernel/concepts/ai-services/chat-completion/?tabs=csharp-AzureOpenAI%2Cpython-AzureOpenAI%2Cjava-AzureOpenAI&pivots=programming-language-python

In [115]:
from semantic_kernel import Kernel
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.prompt_template.prompt_template_config import PromptTemplateConfig

# Initialize the Kernel
kernel = Kernel()

# Configure Azure OpenAI Chat Completion
chat_completion = AzureChatCompletion(
    deployment_name="gpt-4o-mini",
    api_key="",  
    base_url="https://08-01-preview",
    service_id="chatbot"
)

# Register the service
kernel.add_service(chat_completion)

# Define prompt settings
prompt_setting = kernel.get_prompt_execution_settings_from_service_id("chatbot")
prompt_setting.max_tokens = 100
prompt_setting.temperature = 0.5

# Define the prompt template
prompt_text ="what is python"

prompt_template = PromptTemplateConfig(
                   template=prompt_text,
                   name="Test1",
                   template_format="semantic-kernel",
                   execution_settings=prompt_setting)

# Register the prompt function
prompt_function = kernel.add_function(function_name="prompt_function",
                                      plugin_name="semantic_kernel_plugin",
                                      prompt_template_config=prompt_template
     )

# Define an asynchronous invocation
result = await kernel.invoke(prompt_function)
print(result)

Python is a high-level, interpreted programming language known for its readability and simplicity. It was created by Guido van Rossum and first released in 1991. Python supports multiple programming paradigms, including procedural, object-oriented, and functional programming.

Key features of Python include:

1. **Readability**: Python's syntax is designed to be clear and easy to understand, which makes it an excellent choice for beginners.

2. **Dynamically Typed**: Variables in Python do not require an


In [118]:
#result